In [171]:
import pandas as pd
import numpy as np 
%matplotlib inline 

This Ipython notebook was uploaded at PSRC github: https://github.com/psrc/travel-modeling  

In [172]:
# Set which transit agency's data you would like to run
n = 4

In [173]:
path = 'R:/2014_Base_Year/ValidationData/Transit/'

In [174]:
data_set = {}
data_set[1] = 'KC Metro/spring2014_trip_data.csv' # KC Metro
data_set[2] = 'PT/April 2014 Avg Weekeday Boarding Data_PSRC_Request_March 2016.csv' # Pierce Transit
data_set[3] = 'CT/March14Ridership.csv' # Community Transit
data_set[4] = 'KT/PSRC MAR 2014 EST  WEEKDAY.csv' # Kitsap Transit
# WSF(ferries)
# Sound Transit

In [175]:
data = pd.read_csv(path + data_set[n])

# For data has hourly ridership info

In [176]:
if n in [1, 3]:
    data = data[['SignRt', 'Hour_', 'Rides']]
    data_sorted = data.groupby(['SignRt', 'Hour_'], as_index=False)
    data_ridership = data_sorted.aggregate(np.sum)
    data_sorted = data_ridership.pivot(index='SignRt', columns='Hour_', values='Rides').fillna(0)
    # Add route ID 
    data_sorted['Sign_Rt'] = data_sorted.index
    data_sorted['PSRC_Rte_ID'] = data_sorted.apply(lambda row: row['Sign_Rt'] + n*1000, axis=1)
    
    cols = data_sorted.columns.tolist()
    data_table = data_sorted[cols[-2:] + cols[:-2]]
    data_table.index.name = None
    data_table = data_table.reset_index(drop=True)
    data_table.columns
    
    # Attach total row
    data_table = data_table.append(data_table.sum(numeric_only=True), ignore_index=True)
    data_table.iloc[-1:, 0:2] = 'total'

In [177]:
# Reorgnize dataframe 
if n == 3:
    for c in [24, 25, 26, 27]:
        data_table[c] = 0
    print data_table.tail()

# For data was summerized by AM PK, MD, PM PK, EVE, NT

In [178]:
def evenly_distribute(data_col, begin_h, end_h):
    for h in range(begin_h, end_h):
        l = end_h - begin_h
        data[h] = data[data_col].convert_objects(convert_numeric=True)/l
    return data

In [179]:
if n == 2:
    data = data.replace('N/A', 0)
    for rt_content in data['SignRt']:
        if rt_content in ['EAST', 'NORTH', 'SOUTH', 'Totals', 'WEST']:
            data = data[data.SignRt != rt_content]
    print data.columns
if n == 4:
    data = data.drop('NAME', 1)
    data = data.iloc[:-1, :] # Delete the total row, we will create a new one 

In [180]:
if n in [2, 4]:
    data = evenly_distribute(data.columns[1:2], 6, 9)
    data = evenly_distribute(data.columns[2:3], 9, 15)
    data = evenly_distribute(data.columns[3:4], 15, 18)
    data = evenly_distribute(data.columns[4:5], 18, 22)
    data = evenly_distribute(data.columns[5:6], 22, 30)
    data['PSRC_Rte_ID'] = data['SignRt'].convert_objects(convert_numeric=True) + n*1000
    
    # Reorgnize dataframe 
    data.rename(columns={29: 5, 28: 4}, inplace=True)
    cols = ['SignRt', 'PSRC_Rte_ID', 4, 5] + [i for i in range(6, 28)]
    data_table = data[cols]
    data_table = data_table.fillna(0)
    
    # Total row
    data_table = data_table.append(data_table.sum(numeric_only=True), ignore_index=True)
    data_table.iloc[-1:, 0:2] = 'total'

C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Anaconda\lib\site-packages\IPython\kernel\__main__.py:7: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.


#  Save the table

In [181]:
# Validation
# Note: KC metro hour ranges from 4-28, other agencies are 4-27
data_table.tail()

,SignRt,PSRC_Rte_ID,4,5,6,7,8,9,10,11,...,18,19,20,21,22,23,24,25,26,27
39,97,4097,4.375,4.375,15.333333,15.333333,15.333333,0.000000,0.000000,0.000000,...,9.25,9.25,9.25,9.25,4.375,4.375,4.375,4.375,4.375,4.375
40,98,4098,4.125,4.125,11.333333,11.333333,11.333333,0.000000,0.000000,0.000000,...,7.25,7.25,7.25,7.25,4.125,4.125,4.125,4.125,4.125,4.125
41,99,4099,2.750,2.750,13.000000,13.000000,13.000000,0.000000,0.000000,0.000000,...,7.25,7.25,7.25,7.25,2.750,2.750,2.750,2.750,2.750,2.750
42,91,4091,10.250,10.250,33.666667,33.666667,33.666667,1.333333,1.333333,1.333333,...,17.50,17.50,17.50,17.50,10.250,10.250,10.250,10.250,10.250,10.250
43,total,total,98.625,98.625,593.333333,593.333333,593.333333,517.166667,517.166667,517.166667,...,287.50,287.50,287.50,287.50,98.625,98.625,98.625,98.625,98.625,98.625


In [182]:
# Save data to csv
data_table.to_csv(path + str(n) + '.csv')

In [183]:
print 'end'

end
